In [1]:
import sys 
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#
# setup MIDAS env equivalent to source setup.sh for jupyter
# waring rispetto all'sh il $ non definisce varibili globali che vanno definite come strighe
#
DAQ_IN_DIR='/home/cygno/DAQ/online'
%set_env MIDAS_EXPTAB=$DAQ_IN_DIR/exptab
%set_env MIDAS_EXPT_NAME=CYGNUS_RD

%set_env MYDRIVER_DIR=$DAQ_IN_DIR/mydrivers

CAENVME='/usr/local/CAEN/CAENVMELib-2.50'
%set_env CAENVME_INCDIR=$CAENVME/include
%set_env CAENVME_LIBDIR=$CAENVME/lib/x64

CAENHV='/usr/local/CAEN/CAENHVWrapper-5.82'
%set_env CAENHV_INCDIR=$CAENHV/include
%set_env CAENHV_LIBDIR=$CAENHV/lib/x64

%set_env ETHERNET_INCDIR=$DAQ_IN_DIR/mydrivers/ethernet
%set_env CAMERA_INCDIR=/usr/local/dcamsdk4/inc
%set_env CAMERA_LIBDIR=/usr/local/lib


env: MIDAS_EXPTAB=/home/cygno/DAQ/online/exptab
env: MIDAS_EXPT_NAME=CYGNUS_RD
env: MYDRIVER_DIR=/home/cygno/DAQ/online/mydrivers
env: CAENVME_INCDIR=/usr/local/CAEN/CAENVMELib-2.50/include
env: CAENVME_LIBDIR=/usr/local/CAEN/CAENVMELib-2.50/lib/x64
env: CAENHV_INCDIR=/usr/local/CAEN/CAENHVWrapper-5.82/include
env: CAENHV_LIBDIR=/usr/local/CAEN/CAENHVWrapper-5.82/lib/x64
env: ETHERNET_INCDIR=/home/cygno/DAQ/online/mydrivers/ethernet
env: CAMERA_INCDIR=/usr/local/dcamsdk4/inc
env: CAMERA_LIBDIR=/usr/local/lib


In [4]:
# test from jupyter 
import midas.client

"""
A simple example program that connects to a midas experiment,
reads an ODB value, then sets an ODB value.

Expected output is:

```
The experiment is currently stopped
The new value of /pyexample/eg_float is 5.670000
```
"""

if __name__ == "__main__":
    client = midas.client.MidasClient("pytest")

    # Read a value from the ODB. The return value is a normal python
    # type (an int in this case, but could also be a float, string, bool,
    # list or dict).
    state = client.odb_get("/Runinfo/State")

    if state == midas.STATE_RUNNING:
        print("The experiment is currently running")
    elif state == midas.STATE_PAUSED:
        print("The experiment is currently paused")
    elif state == midas.STATE_STOPPED:
        print("The experiment is currently stopped")
    else:
        print("The experiment is in an unexpected run state")

The experiment is currently stopped


In [9]:
client.odb_get("/Runinfo/")
#client.odb_get("/Experiment/MAX_EVENT_SIZE")

OrderedDict([('State', 1),
             ('Online Mode', 1),
             ('Run number', 6633),
             ('Transition in progress', 0),
             ('Start abort', 0),
             ('Requested transition', 0),
             ('Start time', 'Mon Nov 22 08:03:38 2021'),
             ('Start time binary', 1637564618),
             ('Stop time', 'Mon Nov 22 08:05:01 2021'),
             ('Stop time binary', 1637564701)])

In [7]:
client.odb_get("/System/Buffers/SYSTEM")

OrderedDict([('Size', 33554432),
             ('Write pointer', 28689072),
             ('Read pointer', 2904072),
             ('Filled', 25785000),
             ('Filled pct', 76.84528827667236),
             ('Clients',
              OrderedDict([('ODBEdit',
                            OrderedDict([('count_lock', 2),
                                         ('count_sent', 0),
                                         ('bytes_sent', 0),
                                         ('count_write_wait', 0),
                                         ('time_write_wait', '0x00000000'),
                                         ('max_bytes_write_wait', 0),
                                         ('count_read', 0),
                                         ('bytes_read', 0),
                                         ('get_all_flag', False),
                                         ('read_pointer', 0),
                                         ('bytes_used', 0),
                                      

In [15]:
client.disconnect()

In [15]:

#client = midas.client.MidasClient("pytest")

# Define which buffer we want to listen for events on (SYSTEM is the 
# main midas buffer).
buffer_handle = client.open_event_buffer("SYSTEM")

# Request events from this buffer that match certain criteria. In this
# case we will only be told about events with an "event ID" of 14.
request_id = client.register_event_request(buffer_handle)

# If there's an event ready, `event` will contain a `midas.event.Event`
# object. If not, it will be None. If you want to block waiting for an
# event to arrive, you could set async_flag to False.
event = client.receive_event(buffer_handle, async_flag=True)